In [ ]:
!pip install mne
!pip install hdf5storage

In [ ]:
import numpy as np
import mne
import scipy.io as sio
import hdf5storage
import os
import math


In [ ]:
#Test with a single night

path_raw_data = '//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple/S_01/night_1/EEG_raw.mat'
path_cleaned_data = '//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple_cleaned/S_01/night_1/EEG_raw.mat'

print('loading data')
data_raw = hdf5storage.loadmat(path_raw_data)
print("raw data loaded")

#data_cleaned = hdf5storage.loadmat(path_cleaned_data)['data']
#print("clean data loaded")

In [ ]:
print('debug')

In [ ]:

labels = data_raw['chanlabels'].reshape(25).tolist() #channel labels needed to load data with MNE

# A line to first load a nights data into mne, then resample to 250 hz, then get the numpy array out again with ._data
# Data is converted to 32 bit
data = mne.io.RawArray(data_raw['data'], mne.create_info(labels,500)).resample(250)._data.astype("float32")

In [ ]:
!pip install mne
!pip install hdf5storage

In [ ]:
# Function to filter mne raw signal
def filter_signal(raw, l_freq, h_freq, picks=None):
    raw.filter(l_freq, h_freq, picks=picks)
    #raw.plot(block=True) # Plot the mne raw EEG filtered data
    return raw
    
    
# This section processes the data, and creates files inside the original folder structure.
import numpy as np
import mne
import scipy.io as sio
import hdf5storage
import os
import math



clean_data_dir = '//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple_cleaned'
#raw_data_dir = '//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple'
raw_data_dir = '../data'

# Run through all the cleaned EEG files
for subdir, dirs, files in sorted(os.walk(raw_data_dir)):
    for file in files:
        if "EEG" in file and not "250hz" in file:
            try:
                print(os.path.join(subdir, file))
                data_raw = hdf5storage.loadmat(os.path.join(subdir, file))
                print('Raw data loaded')

                # Labels are needed for MNE. Imported, reshaped and converted to list
                labels = data_raw['chanlabels'].reshape(25).tolist() #channel labels needed to load data with MNE
            
                #print(labels)


                # First load a nights data into mne, then resample to 250 hz, then get the numpy array out again with ._data
                # Data is converted to 32 bit
                npdata = np.nan_to_num(data_raw['data'])# change nan to 0 to enable downsampling
                data = mne.io.RawArray(npdata, mne.create_info(labels,500)).resample(250)._data.astype("float32") #This works, but doesn't filter
                #data = mne.io.RawArray(npdata, mne.create_info(labels,500))
                
                #data.plot(block=True) # Plot the mne raw EEG data
                
                #data = filter_signal(data, 0.1, 100, picks=labels).resample(250)._data.astype("float32")
                             
            
            
                #data = mne.filter.resample(data, down = 2)
                print('downsampling complete')
                
                
            

                # Save downsampled data, and add to new file
                np.save(f'{os.path.join(subdir)}/EEG_raw_250hz_unfiltered', data)

                print(f'Done: {subdir}')
            except:
                print("\n \n \nMissed file:")
                print(os.path.join(subdir, file))
                print("\n \n \n________________________________________")
                # Print the exception
                import traceback
                traceback.print_exc()
                print("\n \n \n")
                pass
                

In [ ]:
# This section processes the data, and creates files inside the original folder structure.
import numpy as np
import mne
import scipy.io as sio
import hdf5storage
import os
import math
from scipy import signal


from scipy.fft import fftshift

import matplotlib.pyplot as plt


raw_data_dir = '../data'

# Run through all the cleaned EEG files
for subdir, dirs, files in sorted(os.walk(raw_data_dir)):
    for file in files:
        if "EEG_raw_250hz.npy" in file:
            try:
                print(os.path.join(subdir, file))
                data = np.load(os.path.join(subdir, file))
                print('Time series data loaded')
                
                f, t, Sxx = signal.spectrogram(data[:], fs=250,nperseg=250, noverlap=125)
                print('Spectrogram complete')
                for j in range(Sxx.shape[0]):
                    for i in range(Sxx.shape[1]):
                        Sxx[j][i] = (Sxx[j][i] - Sxx[j][i].mean()) / Sxx[j][i].std()

                #plt.pcolormesh(t[0:10000], f, Sxx[0:10000], shading='gouraud')
                plt.pcolormesh(Sxx[0],norm=plt.Normalize(vmin=0, vmax=1))
                
                
                plt.ylabel('Frequency [Hz]')

                plt.xlabel('Time')

                plt.show()
                
                labels = np.load(os.path.join(subdir, "artefact_annotations.npy"))
                
                plt.plot(labels[0][0:500*250])
                plt.show()
                
                
                
                

                print("\n ________________________________________")
                
            

                # Save downsampled data, and add to new file
                #np.save(f'{os.path.join(subdir)}/spectrogram', data)

                print(f'Done: {subdir}')
            except:
                print("\n \n \nMissed file:")
                print(os.path.join(subdir, file))
                print("\n \n \n________________________________________")
                # Print the exception
                import traceback
                traceback.print_exc()
                print("\n \n \n")
                pass

In [2]:

# Process the spectrograms and save them in the same folder structure
# Create all the segments, and sort them in artefact and clean segments

import numpy as np
import mne
import scipy.io as sio
import hdf5storage
import os
import math
from scipy import signal
from scipy.fft import fftshift
import matplotlib.pyplot as plt
import random

raw_data_dir = '../data'

skips = 0

for subdir, dirs, files in sorted(os.walk(raw_data_dir)):
    for file in files:
        if "EEG_raw_250hz.npy" in file:
            print(os.path.join(subdir))

            if skips >= 18:
                Sxx = np.load(subdir+"/spectrogram.npy")
                annotations = np.load(subdir+"/artefact_annotations.npy")

                print("spectrogram loaded")

                good_segments = []
                bad_segments = []

                for channel_number, channel in enumerate(Sxx):
                    print(channel.shape)
                    for i in range(channel.shape[1]):
                        if (i % 20 == 0) and i < channel.shape[1] - 21:
                            if sum(annotations[channel_number][i*125:i*125+2500]) > 0:
                                bad_segments.append(channel[:,i:i+20])
                            else:
                                good_segments.append(channel[:,i:i+20])
                                
                print("segments created")
                    
                good_segments = random.sample(good_segments,min(len(bad_segments),len(good_segments)))
                
                print("segments shuffled")
                np.save(f'{os.path.join(subdir)}/spectrogram_good_segments', np.array(good_segments))
                np.save(f'{os.path.join(subdir)}/spectrogram_bad_segments', np.array(bad_segments))
            else:
                skips += 1 
                print(f"skipped: {os.path.join(subdir)}")
            




../data/study_1A_mat_simple/S_01/night_1
skipped: ../data/study_1A_mat_simple/S_01/night_1
../data/study_1A_mat_simple/S_01/night_2
skipped: ../data/study_1A_mat_simple/S_01/night_2
../data/study_1A_mat_simple/S_01/night_3
skipped: ../data/study_1A_mat_simple/S_01/night_3
../data/study_1A_mat_simple/S_01/night_4
skipped: ../data/study_1A_mat_simple/S_01/night_4
../data/study_1A_mat_simple/S_02/night_1
skipped: ../data/study_1A_mat_simple/S_02/night_1
../data/study_1A_mat_simple/S_02/night_2
skipped: ../data/study_1A_mat_simple/S_02/night_2
../data/study_1A_mat_simple/S_02/night_3
skipped: ../data/study_1A_mat_simple/S_02/night_3
../data/study_1A_mat_simple/S_02/night_4
skipped: ../data/study_1A_mat_simple/S_02/night_4
../data/study_1A_mat_simple/S_03/night_1
skipped: ../data/study_1A_mat_simple/S_03/night_1
../data/study_1A_mat_simple/S_03/night_2
skipped: ../data/study_1A_mat_simple/S_03/night_2
../data/study_1A_mat_simple/S_03/night_3
skipped: ../data/study_1A_mat_simple/S_03/night_3